# Bigram.py

Bigram 模型是一种简单的语言模型，它基于前一个词（或字符）来预测下一个词（或字符）的概率。具体来说，在给定当前词或字符的情况下，模型通过学习当前字符与下一个字符之间的关系，来预测下一个可能出现的字符。

Bigram 模型会学习如下的字符对：
- "h" -> "e"
- "e" -> "l"
- "l" -> "l"
- "l" -> "o"

训练之后，模型根据前一个字符来预测下一个字符。例如，当输入字符是 "h" 时，模型会预测 "e" 为下一个字符。

Bigram 模型可以扩展为更高阶的 N-gram 模型

Bigram 模型：

"Bi" 在这个词中表示 “2”，即每次考虑两个连续的词或字符来进行建模。Bigram 模型会根据前一个词来预测下一个词。

N-gram 模型：

"N" 在这个词中表示 任意数，即 N 可以是 1、2、3 等等。N-gram 模型表示每次考虑 N 个连续的词或字符来进行建模。

### 导入库

导入了必要的PyTorch模块，包括神经网络模块nn和一些功能性函数F。

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F

### 超参数设置

批量大小（batch_size）、上下文长度（block_size）、训练的最大迭代次数（max_iters）、评估间隔（eval_interval）、学习率（learning_rate）、设备（CPU或GPU），以及评估时的迭代次数（eval_iters）

In [11]:
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

### 数据准备

生成字符集的映射表（stoi 和 itos），用于将字符和整数索引相互转换。

把字符转为int型，为了让计算机理解；计算机输出时，再转为字符，让人类理解

stoi,itos可以理解为字典

In [12]:
with open('./ng-video-lecture/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

### 训练和验证数据集的拆分

数据集分为训练集和验证集，训练集为前90%的数据，验证集为后10%的数据。

In [13]:
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

### 数据加载函数
示例

block_size = 3

batch_size = 2

data = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

ix = [2, 4]

x = [[2, 3, 4],[4, 5, 6]]

y = [[3, 4, 5],[5，6, 7]]

y中与x相对应位置的值，为x应该预测的值，前面预测后面的

In [14]:
# 从训练集或者验证集取数据
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # 把数据放到GPU运行
    x, y = x.to(device), y.to(device)
    return x, y

torch.randint：生成在指定范围内均匀分布的随机整数张量。

torch.randint(low, high, size, out=None, dtype=torch.int64, layout=torch.strided, device=None, requires_grad=False)

    low：生成随机整数的下界（包含该值）。
    high：生成随机整数的上界（不包含该值）。
    size：生成的张量的形状，通常是一个元组，表示每个维度的大小。
    out（可选）：输出的张量。
    dtype（可选）：张量的类型，默认是 torch.int64。
    layout（可选）：张量的布局，默认是 torch.strided。
    device（可选）：张量所在的设备（CPU 或 GPU）。
    requires_grad（可选）：如果为 True，则会为生成的张量记录梯度。

torch.stack 是 PyTorch 中的一个函数，用于沿着一个新维度将一组张量拼接在一起。与 torch.cat 不同的是，torch.stack 会在指定的维度上添加一个新的维度，并在该维度上堆叠输入的张量。

torch.stack(tensors, dim=0, out=None)

    tensors：一个可迭代的张量序列（如列表、元组等）。这些张量的形状必须相同。
    dim（可选）：要在其上插入新维度的位置。默认是 0，表示在第一个维度（批次维度）上堆叠张量。
    out（可选）：输出张量。如果指定了这个参数，那么结果会直接写入这个张量。

In [15]:
import torch

# 创建两个形状为 (3, 4) 的张量
a = torch.tensor([[1, 2, 3, 4],
                  [5, 6, 7, 8],
                  [9, 10, 11, 12]])

b = torch.tensor([[13, 14, 15, 16],
                  [17, 18, 19, 20],
                  [21, 22, 23, 24]])

# 在第0个维度上堆叠
stacked_tensor = torch.stack([a, b], dim=0)
print(stacked_tensor)
stacked_tensor = torch.stack([a, b], dim=1)
print(stacked_tensor)

tensor([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],

        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]])
tensor([[[ 1,  2,  3,  4],
         [13, 14, 15, 16]],

        [[ 5,  6,  7,  8],
         [17, 18, 19, 20]],

        [[ 9, 10, 11, 12],
         [21, 22, 23, 24]]])


### 估计模型损失
定义了一个用于估计训练和验证集损失的函数estimate_loss，在评估模式下对模型进行评估，然后返回平均损失。

@torch.no_grad() 是一个上下文管理器，用于临时禁止梯度计算。
在某些情况下，如模型评估或推理时，我们不需要计算梯度，因为我们只需要前向传播来获得预测结果。使用 @torch.no_grad() 可以节省内存和计算资源。

model.eval() 将模型设置为评估模式；model.train() 将模型设置为训练模式（training mode）。

In [16]:
@torch.no_grad()
def estimate_loss():
    out = {}
    
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out